### Objetivo
* Este projeto visa desenvolver habilidades de limpeza de dados e manipulação de strings.

### Motivação
* No mundo real, a coleta de dados geralmente envolve diferentes origens, como APIs, bancos de dados, planilhas e intervenção humana. Por isso, é provável que esses dados apresentem padrões distintos.

### Explicação:

* Neste projeto, possuo dois conjuntos de dados que contêm nomes de organizações listadas no site da Forbes. No entanto, os nomes das empresas em um arquivo CSV estão na forma como os clientes reconhecem a empresa, enquanto no outro estão os nomes completos das empresas, por exemplo: Apple versus Apple Inc.


* Para um ser humano, identificar que "AECOM" e "AECOM TECHNOLOGY CORPORATION" representam a mesma empresa é fácil. Contudo, como podemos fazer com que a máquina pense de maneira semelhante a um humano?


* Para essa tarefa, pretendo utilizar a distância de Levenshtein, implementada na biblioteca fuzzywuzzy do Python. Essa biblioteca retorna um valor numérico de 0 a 100, representando a similaridade entre duas strings.

In [1]:
import pandas as pd
import itertools
from fuzzywuzzy import fuzz, process

In [2]:
df1 = pd.read_csv("company_names.csv")
df2 = pd.read_csv("full_company_names.csv")

In [3]:
df1.head()

,Company Names
0,JPMorgan Chase
1,Saudi Arabian Oil Company (Saudi Aramco)
2,ICBC
3,China Construction Bank
4,Agricultural Bank of China


In [4]:
df2.sort_values(by=['Full Company Names'], ascending=False).head()

,Full Company Names
105,Zurich Insurance Group Ltd.
16,Wells Fargo & Co.
22,Walmart Inc.
28,Volkswagen AG
121,Visa Inc.


### 1. Crie o dataframe df contendo o produto dos dois CSVs

In [5]:
df = pd.DataFrame(
    itertools.product(df1['Company Names'], df2['Full Company Names']),
    columns=['CSV1', 'CSV 2']
)

In [6]:
df.head()

,CSV1,CSV 2
0,JPMorgan Chase,JPMorgan Chase & Co.
1,JPMorgan Chase,Saudi Arabian Oil Company (Saudi Aramco)
2,JPMorgan Chase,Industrial and Commercial Bank of China Limite...
3,JPMorgan Chase,China Construction Bank Corporation
4,JPMorgan Chase,Agricultural Bank of China Limited


### 2. Crie uma nova coluna "Ratio Score" para guardar a distância entre todas as linhas do dataframe.

In [7]:
test = [fuzz.partial_ratio(csv_1, csv_2) for csv_1, csv_2 in df.values]
test[:10]

[100, 36, 43, 36, 43, 36, 23, 29, 29, 10]

In [8]:
df['Ratio Score'] = [fuzz.partial_ratio(csv_1, csv_2) for csv_1, csv_2 in df.values]

In [9]:
df.head()

,CSV1,CSV 2,Ratio Score
0,JPMorgan Chase,JPMorgan Chase & Co.,100
1,JPMorgan Chase,Saudi Arabian Oil Company (Saudi Aramco),36
2,JPMorgan Chase,Industrial and Commercial Bank of China Limite...,43
3,JPMorgan Chase,China Construction Bank Corporation,36
4,JPMorgan Chase,Agricultural Bank of China Limited,43


### 3. Quantas linhas tem o ratio score maior ou igual a 90?

In [10]:
df.loc[df['Ratio Score'] >= 90].shape

(144, 3)

#### r: 149

### 4. Qual é a empresa correspondente em CSV2 para Mistubishi no CSV1?

In [11]:
df.query("`CSV1` == 'Mitsubishi' and `Ratio Score` > 90")

,CSV1,CSV 2,Ratio Score
11174,Mitsubishi,Mitsubishi Corporation,100
11190,Mitsubishi,"Mitsubishi UFJ Financial Group, Inc.",100


In [12]:
df.query("`CSV1` == 'Mitsubishi UFJ Financial' and `Ratio Score` > 90")

,CSV1,CSV 2,Ratio Score
13590,Mitsubishi UFJ Financial,"Mitsubishi UFJ Financial Group, Inc.",100


#### r: Mitsubishi Corporation

### 5. Qual é a empresa CSV2 correspondente do Santander?

In [13]:
df.loc[
    (df['CSV1'] == "Santander") &
    (df['Ratio Score'] > 90)
]

,CSV1,CSV 2,Ratio Score
7248,Santander,"Banco Santander, S.A.",100


#### r: Banco Santander, S.A.

### 6. CSV1 contém (Porsche Automobil Holding), há correspondência no CSV2?

In [14]:
df.query("`CSV1` == 'Porsche Automobil Holding' and `Ratio Score` > 90")

,CSV1,CSV 2,Ratio Score


#### r: não

### 7. Existe uma empresa correspondente para a Cisco Systems?

In [15]:
df.query("`CSV1` == 'Cisco Systems' and `Ratio Score` > 90")

,CSV1,CSV 2,Ratio Score
13892,Cisco Systems,"Cisco Systems, Inc.",100


#### r: sim